<a href="https://colab.research.google.com/github/jeekyung/ESAA/blob/main/eda_%EC%8B%A4%ED%96%89%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
area = ['강원', '경기', '경남', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']

path = "/content/drive/MyDrive/예보데이터"
file_list = []
for i in range(2012, 2017):
    for a in area:
        file_list.append(glob.glob(path+'/'+str(i)+'/'+str(a)+'/*'))

In [ ]:
rain_fall_list=[]
pre_form_list=[]
rain_prob_list=[]
snow_cov_list=[]
list_x = ['강수량','강수형태','강수확률','신적설']

for i in range(len(file_list)):
  for name in file_list[i]:
    if '강수량' in name:
      rain_fall_list.append(name)
    if '강수형태' in name:
      pre_form_list.append(name)
    if '강수확률' in name:
     rain_prob_list.append(name)
    if '신적설' in name:
      snow_cov_list.append(name)
    else:
      pass

#강수량(rain_fall)

In [ ]:
for name in rain_fall_list:
  if '6시간' in name:
    rain_fall_list.remove(name)
len(rain_fall_list)
# 실행할 때마다 결괏값 달라지다가 163에서 멈춤 -> 왜????

307

In [ ]:
# 강수량 파일 읽기
rain_fall=[]
for name in rain_fall_list:
  rain_fall.append(pd.read_csv(name))

In [ ]:
for i in range(len(rain_fall)):
  rain_fall[i].drop_duplicates(inplace=True)

In [ ]:
# day에 year,month 추가하는 함수
def add_ym_1(df):
  ym = int(df.columns[-1].strip()[-8:-2])
  for j in range(len(df)):
    #값 앞의 빈칸 없애기
    df.iloc[j,0]=df.iloc[j,0].strip()
    #'Start' 있으면 year,month 변경
    if 'Start' in df.iloc[j,0]:
      ym = int(df.iloc[j,0].strip()[-8:-2])
    df.iloc[j,0] = str(ym)+str(df.iloc[j,0]).zfill(2)

In [ ]:
for i in range(len(rain_fall)):
  # year,month 추가
  add_ym_1(rain_fall[i])
  #'Start' drop
  rain_fall[i].dropna(inplace=True)

In [ ]:
rain_fall[0].head()

,format..day,hour,forecast,value..location.87_142.Start...20120101
0,20120101,200.0,4.0,10.0
1,20120101,200.0,7.0,5.0
2,20120101,200.0,10.0,0.0
3,20120101,200.0,13.0,0.0
4,20120101,500.0,4.0,10.0


In [ ]:
'''
for i in range(len(rain_fall)):
  if rain_fall[i]['forecast'].dtype != float:
    rain_fall[i]['forecast'] = rain_fall[i]['forecast'].astype(float)
'''

In [ ]:
for i in range(len(rain_fall)):
    rain_fall[i] = rain_fall[i][rain_fall[i]['forecast'] == rain_fall[i]['forecast'].min()]

In [ ]:
for i in range(len(rain_fall)):
  rain_fall[i].reset_index(drop=True,inplace=True)

In [ ]:
'''
for i in range(len(rain_fall)):
  rain_fall[i]['hour']=rain_fall[i]['hour'].astype(int).astype(str).str.zfill(4)
  rain_fall[i]['hour']=rain_fall[i]['hour'] + '00'
  rain_fall[i].iloc[:,0] = pd.to_datetime(rain_fall[i].iloc[:,0]+rain_fall[i]['hour'])
'''

In [ ]:
for i in range(len(rain_fall)):
  # 'format..day' 변수 datetime으로
  rain_fall[i].iloc[:,0] = pd.to_datetime(rain_fall[i].iloc[:,0],format='%Y%m%d')
  #'hour' 변수 datetime으로
  rain_fall[i]['hour']=rain_fall[i]['hour'].astype(int).astype(str).str.zfill(4)
  rain_fall[i]['hour']=rain_fall[i]['hour'] + '00'
  rain_fall[i]['hour'] = pd.to_datetime(rain_fall[i]['hour'],format='%H%M%S')

In [ ]:
from datetime import timedelta
for i in range(len(rain_fall)):
  for j in range(len(rain_fall[i])):
    rain_fall[i].loc[j,'time'] = rain_fall[i].loc[j,'hour']+timedelta(hours=rain_fall[i].loc[j,'forecast'])

TypeError: ignored

In [ ]:
rain_fall[0].head()

,format..day,hour,forecast,value..location.87_142.Start...20120101,time
0,2012-01-01,1900-01-01 02:00:00,4.0,10.0,1900-01-01 06:00:00
1,2012-01-01,1900-01-01 05:00:00,4.0,10.0,1900-01-01 09:00:00
2,2012-01-01,1900-01-01 08:00:00,4.0,10.0,1900-01-01 12:00:00
3,2012-01-01,1900-01-01 11:00:00,4.0,10.0,1900-01-01 15:00:00
4,2012-01-01,1900-01-01 14:00:00,4.0,5.0,1900-01-01 18:00:00


In [ ]:
rain_fall_list[0][39:41]

'강원'

In [ ]:
# region col 추가
for i,name in enumerate(rain_fall_list):
  rain_fall[i]=rain_fall[i].assign(region=name[39:41])
rain_fall[0].head()

,format..day,hour,forecast,value..location.87_142.Start...20120101,time,region
0,2012-01-01,1900-01-01 02:00:00,4.0,10.0,1900-01-01 06:00:00,강원
1,2012-01-01,1900-01-01 05:00:00,4.0,10.0,1900-01-01 09:00:00,강원
2,2012-01-01,1900-01-01 08:00:00,4.0,10.0,1900-01-01 12:00:00,강원
3,2012-01-01,1900-01-01 11:00:00,4.0,10.0,1900-01-01 15:00:00,강원
4,2012-01-01,1900-01-01 14:00:00,4.0,5.0,1900-01-01 18:00:00,강원


In [ ]:
for i in range(len(rain_fall)):
  rain_fall[i] = rain_fall[i][rain_fall[i]['format..day'].hour==12 | rain_fall[i]['format..day'].hour==0]
#시간이 12:00이거나 00:00인 애들의 평균값

In [ ]:
for i in range(len(rain_fall)):
  print(len(rain_fall[i]))

In [ ]:
for i in range(len(rain_fall)):
  rain_fall[i].iloc[:,]=rain_fall[i].loc[:,].resample('1D').mean()

# 강수 형태(pre_form)

In [ ]:
pre_form_list

In [ ]:
pre_form=[]
for name in pre_form_list:
  pre_form.append(pd.read_csv(name))

In [ ]:
pre_form[0]

In [ ]:
for i in range(len(pre_form)):
  print(pre_form[i].isnull().sum())

In [ ]:
for i in range(len(pre_form)):
  if pre_form[i].iloc[:,-1].isnull().sum() > 11:
    print(pre_form[i][pre_form[i]['forecast']==na])

In [ ]:
pre_form[0].iloc[:,-1].value_counts()

#강수 확률(rain_prob)

In [ ]:
rain_prob_list

['/content/drive/MyDrive/예보데이터/2012/강원/토성면_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/신북읍_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/갈말읍_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/대관령면_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/신사우동_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/사천면_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/중앙동_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/송정동_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/명륜1동_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/영월읍_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/인제읍_강수확률_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/홍천읍

In [ ]:
rain_prob=[]
for name in rain_prob_list:
  rain_prob.append(pd.read_csv(name))

In [ ]:
for i in range(len(rain_prob)):
  print(rain_prob[i].isnull().sum())

#신적설(snow_cov)

In [ ]:
snow_cov_list

['/content/drive/MyDrive/예보데이터/2012/강원/토성면_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/신북읍_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/갈말읍_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/대관령면_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/신사우동_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/사천면_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/중앙동_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/송정동_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/명륜1동_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/영월읍_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/예보데이터/2012/강원/인제읍_12시간신적설_201201_201212.csv'

In [ ]:
snow_cov=[]
for name in snow_cov_list:
  snow_cov.append(pd.read_csv(name))

In [ ]:
for i in range(len(snow_cov)):
  print(snow_cov[i].isnull().sum())

format..day                                 0
hour                                       11
forecast                                   11
value..location.87_142.Start...20120101    11
dtype: int64
format..day                                 0
hour                                       11
forecast                                   11
value..location.73_135.Start...20120101    11
dtype: int64
format..day                                 0
hour                                       11
forecast                                   11
value..location.65_139.Start...20120101    11
dtype: int64
format..day                                 0
hour                                       11
forecast                                   11
value..location.89_130.Start...20120101    11
dtype: int64
format..day                                 0
hour                                       11
forecast                                   11
value..location.73_134.Start...20120101    11
dtype: int64
format..day    